### Civilization VI District Optimization Model (Terrain Adjacency Focus)

##### **1. Introduction**
The objective of this model is to determine the optimal placement of city districts within a defined city radius (e.g., 3 tiles, potentially represented by a 7x7 grid/matrix) around a pre-determined City Center tile. The goal is to maximize the total yield generated from adjacency bonuses provided by terrain features, specific resources, and adjacency to the City Center itself. This initial version explicitly excludes adjacency bonuses that districts provide to each other.

##### **2. Data Sets**
$\text{Tiles}$: The set of all tile coordinates $(x,y)$ within the defined city radius.  

$(x_{cc}​,y_{cc}​) \in \text{Tiles}$: The specific coordinates of the pre-determined City Center tile.  

$\text{Tiles}_{\text{Workable}} \subset \text{Tiles}$: The subset of tiles within the radius where districts can potentially be built (i.e., "workable" tiles). This excludes the City Center tile $(x_{cc}, y_{cc})$ and tiles with impassable terrain (like Mountains, unless buildable by specific civs/conditions), or water tiles unless suitable for specific districts (like Harbor).
- Note: The exact definition of $\text{Tiles}_{\text{Workable}}$​ depends on game rules and potential civilization-specific abilities.  

$\text{Districts}$: The set of all possible district types considered (e.g., Campus, Holy Site, Commercial Hub, Harbor, Aqueduct, Entertainment Complex, Water Park, Industrial Zone, etc.).  

$\text{Districts}_{\text{Unique}} \subseteq \text{Districts}$: The subset of districts that are unique per city (i.e., only one of each can be built). Most specialty districts belong here.  

$\text{Districts}_{\text{Mutually Exclusive}}$​: A collection of subsets of $\text{Districts}$, where each subset contains districts that are mutually exclusive with each other.
Example: $\text{Entertainment Complex}, \text{Water Park} \in \text{Districts}_{\text{Mutually Exclusive}}$.  

$\text{Features}$: The set of relevant terrain features, resources, or specific tile types that provide adjacency bonuses (e.g., Mountain, River, Coast, Reef, Geothermal Fissure, Rainforest, Woods, Natural Wonder, Strategic Resource, Luxury Resource, etc.).

#### **3. Input Data**

*These values are pre-determined based on the chosen City Center location and the surrounding map data.*  

$Adj(x,y,x′,y′)$: Binary parameter. 1 if tile $(x,y)\in \text{Tiles}$ and tile $(x′,y′)\in \text{Tiles}$ are adjacent according to the game's hex grid rules, 0 otherwise.  

$HasFeature(x′,y′,f)$: Binary parameter. 1 if tile $(x′,y′)\in \text{Tiles}$ possesses the feature or characteristic $f \in \text{Features}$, 0 otherwise.
- Refinement Note: For features like Rivers, decide if f represents the river tile itself or if you use a parameter like $IsAdjacentToRiver(x,y)$ for buildable tiles next to a river. The latter is often easier for bonus calculation. Let's assume $HasFeature(x′,y′,River)$ means tile $(x′,y′)$ is a river tile.  

$TerrainBonusValue(d,f)$: Numerical parameter. The base yield bonus value that district $d \in \text{Districts}$ receives for being adjacent to one instance of feature $f \in \text{Features}$ on an adjacent tile. (e.g., $TerrainBonusValue(\text{Campus},\text{Mountain})=+1 \text{Science}$). This value should represent the weighted importance if optimizing across multiple yields, or the specific yield value if optimizing for one type (e.g., just Science).  

$CCBonusValue(d)$: Numerical parameter. The base yield bonus value district $d \in \text{District}$ receives for being adjacent to the City Center tile $(x_{cc}​,y_{cc}​)$. (e.g., $CCBonusValue(\text{Commercial Hub})=+1 \text{Gold}$). Often 0 for many districts.  

$IsAdjacentToCC(x,y)$: Binary parameter. 1 if $Adj(x,y,x_{cc}​,y_{cc}​)=1$, 0 otherwise. (Can be pre-calculated).  

$IsAdjacentToFreshwater(x,y)$: Binary parameter. 1 if tile $(x,y)$ is adjacent to a tile $(x′,y′)$ which is a River, Lake, Oasis, or Mountain. (Can be pre-calculated).  

$IsCoast(x,y)$: Binary parameter. 1 if tile $(x,y)$ is a Coast tile.

$IsAdjacentToLand(x,y)$: Binary parameter. 1 if tile $(x,y)$ is adjacent to at least one land tile $(x′,y′)$. (Can be pre-calculated).

#### **4. Decision Variables**

$Build(x,y,d)$: Binary variable.
- $Build(x,y,d)=1$ if district $d \in \text{Districts}$ is built on workable tile $(x,y)\in \text{Tiles}_{\text{Workable}}$​.
- $Build(x,y,d)=0$ otherwise.

#### **5. Objective Function**
Maximize the total weighted adjacency yield derived from adjacent terrain features/resources and adjacency to the City Center.
$$
\begin{gathered}
\text{Max} \quad \sum_{(x,y) \in \text{Tiles}_{\text{Workable}}} \sum_{d \in \text{Districts}} Build(x, y, d) \times \\
 \left( \left( \sum_{(x',y') | Adj(x,y,x',y')} \sum_{f \in Features} HasFeature(x', y', f) \times TerrainBonusValue(d, f) \right) \right. \\
 \left. + IsAdjacentToCC(x,y) \times CCBonusValue(d) \right)
\end{gathered}
$$
Explanation:
- The outer sums iterate through every possible district placement on every workable tile.
- The term contributes to the sum only if $Build(x,y,d)=1$.
- The first inner part $ \sum_{(x',y') | Adj(x,y,x',y')} \dots $ sums the bonuses district $d$ at $(x,y)$ gets from all relevant features f located on all adjacent tiles $(x′,y′)$.
- The second inner part $ IsAdjacentToCC(x,y) \times CCBonusValue(d) $ adds the bonus if the district is placed next to the City Center.

#### **6. Constraints**
**One District Per Workable Tile: Each workable tile can host at most one district.**  
$$ \sum_{d \in \text{Districts}} Build(x, y, d) \leq 1 \quad \forall (x, y) \in \text{Tiles}_{\text{Workable}} $$

**Unique District Limit: Each district type designated as unique can only be built once within the city's radius.**
$$ \sum_{(x,y) \in \text{Tiles}_{\text{Workable}}} Build(x, y, d) \leq 1 \quad \forall d \in \text{Districts}_{\text{Unique}} $$  

**Mutual Exclusivity: For each set MES​et of mutually exclusive districts, at most one district from that set can be built.**
$$ \sum_{d \in \text{Mutually Exclusive}_{\text{Set}}} \sum_{(x,y) \in \text{Tiles}_{\text{Workable}}} Build(x, y, d) \leq 1 \quad \forall \text{Mutually Exclusive}_{\text{Set}} \in \text{Districts}_{\text{Mutually Exclusive}} $$  
###### (Example: For $\text{Mutually Exclusive}_{\text{Set}}=\text{Entertainment Complex},\text{Water Park}$ this becomes: $ \sum_{(x,y) \in \text{Tiles}_{\text{Workable}}} Build(x, y, \text{Entertainment Complex}) + \sum_{(x,y) \in \text{Tiles}_{\text{Workable}}} Build(x, y, \text{Water Park}) \leq 1 $)

**Aqueduct Placement Requirements: An Aqueduct can only be built if the tile (x,y) is adjacent to the City Center AND adjacent to a freshwater source.**
$$ Build(x, y, \text{Aqueduct}) \leq IsAdjacentToCC(x, y) \quad \forall (x, y) \in \text{Tiles}_{\text{Workable}} $$
$$ Build(x, y, \text{Aqueduct}) \leq IsAdjacentToFreshwater(x, y) \quad \forall (x, y) \in \text{Tiles}_{\text{Workable}} $$  

###### (These constraints force $Build(x,y,\text{Aqueduct})$ to be 0 if the conditions axren't met. The tile $(x,y)$ must also be in $\text{Tiles}_{\text{Workable}}$​).  

**Harbor Placement Requirements: A Harbor can only be built if the tile (x,y) is a Coast tile AND is adjacent to a land tile.**   

$$ Build(x, y, \text{Harbor}) \leq IsCoast(x, y) \quad \forall (x, y) \in \text{Tiles}_{\text{Workable}} $$
$$ Build(x, y, \text{Harbor}) \leq IsAdjacentToLand(x, y) \quad \forall (x, y) \in \text{Tiles}_{\text{Workable}} $$
###### (The tile $(x,y)$ must also be in $\text{Tiles}_{\text{Workable}}$​, meaning coast tiles suitable for harbors must be considered workable).  

###### (Optional) Add Other Specific Placement Rules: Include similar constraints for districts like Dams, Canals, etc., based on their specific geographical requirements (e.g., must be on a river tile, must span two landmasses, etc.).




In [12]:
import json
import pandas as pd
from gurobipy import Model, GRB, quicksum
from collections import deque

In [13]:
# CIV 6 DISTRICT PLACEMENT OPTIMIZATION MODEL
# INPUT SETS / DATA INITALIZATION

# parse input data from civ_map_data.json into a pandas dataframe
with open('civ_map_data.json') as f:
    data = json.load(f)
df = pd.DataFrame(data['tiles'])
print(df.head())

# assume that the city center is selected at the tile with the highest normalized score.
best_tile = df.loc[df['normalized_score'].idxmax()]

cc_x = best_tile['x']
cc_y = best_tile['y']

# print(f"City Center selected at ({cc_x}, {cc_y})")

city_center_coords = (cc_x, cc_y)
print(f"City Center coordinates: {city_center_coords}")

# print(city_center_coords)
# print(best_tile)

# GAME CONSTANTS
# Set of Districts to consider
ALL_DISTRICTS = [
    "Campus", "Holy Site", "Harbor", "Government Plaza", "Theater Square", "Entertainment Complex", "Commercial Hub", "Industrial Zone", "Aqueduct", "Water Park", "Dam", "Canal"
]

DISTRICTS_UNIQUE = [
    "Campus", "Holy Site", "Harbor", "Government Plaza", "Theater Square", "Entertainment Complex", "Commercial Hub", "Industrial Zone", "Aqueduct", "Water Park", "Dam", "Canal"
]

DISTRICTS_MUTUALLY_EXCLUSIVE = [
    {"Entertainment Complex", "Water Park"}
]

FEATURES = {
    "Mountain": ["TERRAIN_GRASS_MOUNTAIN", "TERRAIN_DESERT_MOUNTAIN", "TERRAIN_PLAINS_MOUNTAIN"], "Coast": ["TERRAIN_COAST"], "River": ["River"],
    "Lake": ["TERRAIN_LAKE"], "Oasis": ["TERRAIN_DESERT_OASIS"],
    "Geothermal Fissure": ["FEATURE_GEOTHERMAL_FISSURE"], "Reef": ["FEATURE_REEF"],
    "Woods": ["FEATURE_WOODS"], "Rainforest": ["FEATURE_JUNGLE"], "Marsh": ["FEATURE_MARSH"],
    "Natural Wonder": ["NaturalWonder"], "Strategic": ["StrategicResource"], "Luxury": ["LuxuryResource"]
}

# NOTE: NOT ALL BONUSES INCLUDED
# NAMELY HARBOR BONUSES AND INDUSTRIAL ZONE BONUSES
TERRAIN_BONUS_VALUE = {
    ("Campus", "Mountain"): 1, ("Campus", "Reef"): 2, ("Campus", "Geothermal Fissure"): 2,
    ("Campus", "Rainforest"): 0.5, ("Holy Site", "Mountain"): 1, ("Holy Site", "Woods"): 0.5,
    ("Holy Site", "Natural Wonder"): 2, ("Commercial Hub", "River"): 2,
    ("Theater Square", "Wonder"): 2,
    ("Commercial Hub", "River"): 2
}

CC_BONUS_VALUE = { "Harbor": 2 }
for dist in ALL_DISTRICTS: CC_BONUS_VALUE.setdefault(dist, 0)
RADIUS = 3

IMPASSABLE_TERRAIN = {
    "TERRAIN_OCEAN", "TERRAIN_GRASS_MOUNTAIN", "TERRAIN_PLAINS_MOUNTAIN", "TERRAIN_DESERT_MOUNTAIN",
    "TERRAIN_TUNDRA_MOUNTAIN", "TERRAIN_SNOW_MOUNTAIN", "TERRAIN_LAKE",
}
IMPASSABLE_FEATURES = { "FEATURE_ICE" }

# HELPER FUNCTIONS FOR HEX GRID CALCULATION
def get_adjacent_tiles(x, y, all_map_coords_set):
    """
    Gets coordinates of adjacent tiles based on JavaScript 'odd-r' flat-top logic.
    Row parity (y % 2) determines offsets. Corresponds to JS 'getDirectNeighbors'.
    Input and Output coordinates are (x, y).
    """
    x = int(x) # Ensure integer types
    y = int(y)
    direct_offsets = []
    is_even_row = y % 2 == 0

    # Counter-clockwise, starting from the rightmost adjacent tile
    if is_even_row:
        # Offsets for EVEN rows (y=0, 2, 4...)
        direct_offsets = [ [+1,  0], [+1, +1], [ 0, +1], [-1,  0], [ 0, -1], [+1, -1] ]
    else:
        # Offsets for ODD rows (y=1, 3, 5...)
        direct_offsets = [ [+1,  0], [ 0, +1], [-1, +1], [-1,  0], [-1, -1], [ 0, -1] ]

    adjacent = set()
    for dx, dy in direct_offsets:
        neighbor_coords = (x + dx, y + dy)
        # Check if this coordinate exists in the set of all map coordinates
        if neighbor_coords in all_map_coords_set:
            adjacent.add(neighbor_coords)
    return adjacent

def get_tiles_in_radius_bfs(start_coords, max_rings, all_map_coords_set):
    """
    Finds all tiles within max_rings steps from start_coords using BFS.
    Uses the get_adjacent_tiles function based on JS 'odd-r' logic.
    Returns a set of reachable (x, y) coordinates INCLUDING the start tile.
    Corresponds to JS 'getNeighborCoords' but includes center tile in result set.
    """
    start_x, start_y = int(start_coords[0]), int(start_coords[1])
    start_coords_int = (start_x, start_y)

    if start_coords_int not in all_map_coords_set:
        print(f"Warning: Start coordinates {start_coords_int} not in map data.")
        return set()
    if max_rings <= 0:
        return {start_coords_int} # Return only center if radius is 0 or less

    queue = deque([(start_coords_int, 0)]) # Queue holds ((x, y), distance)
    visited = {start_coords_int}
    # *** Start with the center tile included ***
    reachable_tiles = {start_coords_int}

    while queue:
        current_coords, current_dist = queue.popleft()

        # Explore neighbors only if we haven't reached the max distance
        if current_dist < max_rings:
            neighbors = get_adjacent_tiles(current_coords[0], current_coords[1], all_map_coords_set)

            for neighbor in neighbors:
                if neighbor not in visited:
                    visited.add(neighbor)
                    reachable_tiles.add(neighbor) # Add neighbor to results
                    # Add neighbor to queue for further exploration
                    queue.append((neighbor, current_dist + 1))

    return reachable_tiles

# --- Pre-calculation Steps ---

# Create a set of all valid (x, y) coordinates from the DataFrame for quick lookups
# Ensure coordinates are integers for consistency
all_map_coords = set((int(coord[0]), int(coord[1])) for coord in df[['x', 'y']].values)

# Tiles: Set of tiles within the defined radius using BFS
tiles_in_radius = get_tiles_in_radius_bfs(city_center_coords, RADIUS, all_map_coords)
print(f"Total tiles in radius {RADIUS} (using BFS): {len(tiles_in_radius)}")
if not tiles_in_radius:
     print("ERROR: No tiles found within radius (or CC invalid).")
     exit()

# Create a mapping from (x, y) tuple to the tile's data (only for tiles in radius)
# Ensure lookup keys are integers
tile_data = {
    (int(row['x']), int(row['y'])): row for index, row in df.iterrows()
    if (int(row['x']), int(row['y'])) in tiles_in_radius
}

# Tiles_Workable: Subset of tiles_in_radius where districts can potentially be built
tiles_workable = set()
for coords in tiles_in_radius:
    if coords == city_center_coords: continue
    if coords not in tile_data: continue # Should not happen if BFS is correct, but safe check
    tile_info = tile_data[coords]
    terrain = tile_info['terrain']; feature = tile_info['feature']
    is_impassable = (terrain in IMPASSABLE_TERRAIN or (feature is not None and feature in IMPASSABLE_FEATURES))
    coast_terrain_val = FEATURES.get("Coast", ["TERRAIN_COAST"])
    is_workable_coast = terrain in coast_terrain_val
    if not is_impassable or is_workable_coast:
        tiles_workable.add(coords) # Coords are already int tuples
print(f"Workable tiles in radius: {len(tiles_workable)}")

# --- Pre-calculation of Boolean Maps & Adjacency ---

# Adjacency Map: Maps (x, y) -> set of adjacent (x', y') within the radius
adjacency_map = {
    coords: get_adjacent_tiles(coords[0], coords[1], all_map_coords).intersection(tiles_in_radius)
    for coords in tiles_in_radius # Use all tiles in radius for full map context
}

# Features on Tile Map: features_on_tile[(x,y)] -> set of feature names
features_on_tile = {}
# River Adjacency Map: is_adjacent_to_river[(x,y)] -> boolean (Needs specific logic)
# This part is CRUCIAL and depends heavily on how river data is stored in your JSON
# Example Placeholder Logic (assuming 'rivers' field indicates adjacency):
# river_adj_map = {} # Precompute river adjacency if possible from df['rivers']
# ... logic to populate river_adj_map ...
# has_river_edge: Maps (x, y) -> boolean, True if the tile has any river edges defined
has_river_edge = {}
for coords, tile_info in tile_data.items():
    # Check if the 'rivers' key exists and its value is not None and not an empty string
    # for no rivers (e.g., null, NaN, empty list). pandas might load nulls as None or NaN.
    if pd.notna(tile_info.get('rivers')) and tile_info.get('rivers') != "":
        has_river_edge[coords] = True
    else:
        has_river_edge[coords] = False

print(f"Tiles with river edges identified: {sum(has_river_edge.values())}")
# Optional: Print which tiles have rivers for debugging
print({k: v for k, v in has_river_edge.items() if v})

for coords in tiles_in_radius:
    if coords not in tile_data: continue # Skip if somehow not in tile_data
    tile_info = tile_data[coords]
    tile_features = set()
    terrain = tile_info['terrain']; feature = tile_info.get('feature'); resource = tile_info.get('resource'); res_type = tile_info.get('resourcetype', 'default')

    # Map terrain/feature names from FEATURES constant
    for f_name, f_data_vals in FEATURES.items():
        # Handle special cases like River separately if needed
        if f_name == "River": continue
        if f_name == "NaturalWonder": continue # Handle based on specific wonder data if available
        if f_name == "Wonder": continue # Handle based on specific wonder data if available
        if f_name == "Strategic": continue # Handle based on resource type
        if f_name == "Luxury": continue # Handle based on resource type

        if terrain in f_data_vals or feature in f_data_vals:
            tile_features.add(f_name)

    # Add River feature if tile is adjacent to a river (NEEDS YOUR DATA LOGIC)
    # if river_adj_map.get(coords, False):
    #    tile_features.add("River")
    # Simplified: Check if any adjacent tile *is* a river terrain/feature (less accurate for bonuses)
    is_adj_river_terrain = False
    for adj_coords in adjacency_map.get(coords, set()):
        if adj_coords in tile_data:
             # Check if adjacent tile itself is designated as river terrain/feature
             # This is NOT the standard Civ 6 adjacency rule (bonus for district next to river)
             # You likely need to check df['rivers'] or similar property
             pass # Replace with your river adjacency logic

    # Add resource types as features
    if resource:
        if res_type == 'strategic': tile_features.add("Strategic")
        elif res_type == 'luxury': tile_features.add("Luxury")
        # Add bonus resource type if needed

    # Add Natural Wonder feature if applicable (needs specific check)
    # if tile_info.get('is_natural_wonder', False): tile_features.add("Natural Wonder")

    # Add built Wonder feature if applicable (needs specific check)
    # if tile_info.get('is_wonder', False): tile_features.add("Wonder")

    features_on_tile[coords] = tile_features


# isAdjacentToCC: Check adjacency for workable tiles
is_adjacent_to_cc = {
    coords: city_center_coords in adjacency_map.get(coords, set())
    for coords in tiles_workable
}

# isAdjacentToFreshwater: Check adjacency for workable tiles
# Freshwater sources: River, Lake, Oasis, Mountain (Mountains provide fresh water for Aqueducts)
freshwater_feature_names = {"River", "Lake", "Oasis", "Mountain"} # Use names from FEATURES keys
is_adjacent_to_freshwater = {}
for coords in tiles_workable:
    adj_fresh = False
    for adj_coords in adjacency_map.get(coords, set()):
        # Check features on the adjacent tile
        if adj_coords in features_on_tile:
            adj_features = features_on_tile[adj_coords]
            if any(f in freshwater_feature_names for f in adj_features):
                adj_fresh = True
                break
    is_adjacent_to_freshwater[coords] = adj_fresh

# isCoast: Check terrain for workable tiles
coast_terrain_name = FEATURES.get("Coast", ["TERRAIN_COAST"])
is_coast = {
    coords: tile_data[coords]['terrain'] in coast_terrain_name
    for coords in tiles_workable if coords in tile_data # Ensure coords exist
}

# isAdjacentToLand: Check adjacency for workable tiles (primarily for Harbor)
land_terrains = { t for t in df['terrain'].unique() if t not in {"TERRAIN_OCEAN", "TERRAIN_COAST", "TERRAIN_LAKE"} } # Define land terrain types
is_adjacent_to_land = {}
for coords in tiles_workable:
    # Only relevant if the tile itself is Coast (or potentially Lake if allowed)
    if is_coast.get(coords, False): # Add 'or is_lake.get(coords, False)' if lakes are buildable
        adj_land = False
        for adj_coords in adjacency_map.get(coords, set()):
            if adj_coords in tile_data and tile_data[adj_coords]['terrain'] in land_terrains:
                adj_land = True
                break
        is_adjacent_to_land[coords] = adj_land
    else:
        is_adjacent_to_land[coords] = False # Not relevant if the tile itself isn't Coast/Lake

print("Pre-calculations finished.")

    x   y               terrain         feature           resource  \
0  56  25   TERRAIN_GRASS_HILLS  FEATURE_FOREST  RESOURCE_TRUFFLES   
1  11  23  TERRAIN_PLAINS_HILLS  FEATURE_JUNGLE               None   
2  45  31  TERRAIN_PLAINS_HILLS            None     RESOURCE_IVORY   
3  52  27   TERRAIN_GRASS_HILLS            None     RESOURCE_STONE   
4  17  18        TERRAIN_PLAINS  FEATURE_JUNGLE     RESOURCE_COCOA   

  resourcetype           continent rivers  appeal  goodyhut  normalized_score  \
0         None   CONTINENT_PANGAEA  0|4|5     3.0     False             319.0   
1         None  CONTINENT_COLUMBIA    0|5     4.0     False             271.0   
2         None   CONTINENT_PANGAEA    3|4     4.0     False             261.0   
3         None   CONTINENT_PANGAEA      4     5.0     False             261.0   
4         None  CONTINENT_COLUMBIA      3     1.0     False             251.0   

  tier  
0    S  
1    S  
2    S  
3    S  
4    S  
City Center coordinates: (56, 25)
Tota

In [14]:
# DEBUG: Print tiles_workable right before model creation
print(f"Tiles considered workable before creating variables ({len(tiles_workable)}): {sorted(list(tiles_workable))}")
if (57, 24) in tiles_workable: print("DEBUG: (57, 24) IS in tiles_workable before addVars.")
else: print("DEBUG: (57, 24) is NOT in tiles_workable before addVars.")

model = Model("Civ6_District_Optimization")
model.setParam('OutputFlag', 1) # Set to 0 to suppress Gurobi console output

# --- MODIFIED: Explicit Variable Creation ---
# Decision Variables: Build(x, y, d)
# Create variables explicitly using a loop instead of model.addVars
build_vars = {}
print("Creating variables explicitly...")
for coords in tiles_workable:
    for d in ALL_DISTRICTS:
        # Create a unique variable name
        var_name = f"Build[{coords[0]},{coords[1]},{d}]"
        # Add the variable to the model and store it in the dictionary
        build_vars[coords, d] = model.addVar(vtype=GRB.BINARY, name=var_name)
# --- End Modification ---

print(f"Gurobi variables defined ({len(build_vars)} variables).")

# DEBUG: Check if the specific key exists *after* explicit addVar loop
try:
    problem_key = ((57, 24), 'Water Park')
    if problem_key in build_vars:
            print(f"DEBUG: Key {problem_key} FOUND in build_vars after explicit creation.")
    else:
            # This check should ideally not be needed if the loops are correct,
            # but kept for thoroughness.
            if problem_key[0] in tiles_workable and problem_key[1] in ALL_DISTRICTS:
                print(f"DEBUG: Key {problem_key} components ARE in workable/districts lists, but key MISSING from build_vars.")
            else:
                print(f"DEBUG: Key {problem_key} components NOT in workable/districts lists.")

except Exception as e:
    print(f"DEBUG: Error checking key {problem_key}: {e}")


# Objective Function Coefficients
obj_coeffs = {}
# Define the "River" feature key used in TERRAIN_BONUS_VALUE
RIVER_FEATURE_KEY = "River" # Make sure this matches the key in TERRAIN_BONUS_VALUE
print("***"*10)
for coords in tiles_workable: # coords are int tuples
    for d in ALL_DISTRICTS:
        # Sum terrain/feature bonuses from adjacent tiles (Existing Logic)
        terrain_bonus = sum(
            TERRAIN_BONUS_VALUE.get((d, f), 0)
            # IMPORTANT: Exclude the "River" key here if it's in TERRAIN_BONUS_VALUE,
            # as we are handling it separately based on the *placement tile*.
            for adj_coords in adjacency_map.get(coords, set()) # Neighbors of the district tile
            if adj_coords in features_on_tile # Ensure neighbor exists in feature map
            for f in features_on_tile.get(adj_coords, set()) if f != RIVER_FEATURE_KEY # Exclude river check here
        )

        print(f"DEBUG: Terrain bonus for {coords} and district {d}: {terrain_bonus}")

        # Add City Center bonus if adjacent (Existing Logic)
        cc_bonus = CC_BONUS_VALUE.get(d, 0) if is_adjacent_to_cc.get(coords, False) else 0

        # --- NEW: Add River Adjacency Bonus ---
        # Check if the *tile the district is placed on* has a river edge
        river_bonus = 0
        if has_river_edge.get(coords, False):
             # Check if *this district* (d) gets a bonus from rivers
             river_bonus = TERRAIN_BONUS_VALUE.get((d, RIVER_FEATURE_KEY), 0)
        # --- End of New River Bonus Logic ---

        # Combine all bonuses
        total_bonus = terrain_bonus + cc_bonus + river_bonus
        obj_coeffs[coords, d] = total_bonus

# Set Objective Function
# --- MODIFIED: Use quicksum with the explicitly created dictionary ---
model.setObjective(quicksum(build_vars[key] * obj_coeffs.get(key, 0)
                            for key in build_vars), # Iterate through keys in build_vars dict
                    GRB.MAXIMIZE)
# --- End Modification ---
print("Gurobi objective function defined.")
print("***"*10)

# --- Constraints ---
print("Defining constraints...")
# 1. One District Per Workable Tile
# --- MODIFIED: Use build_vars dictionary access ---
for coords in tiles_workable:
    model.addConstr(quicksum(build_vars[coords, d] for d in ALL_DISTRICTS if (coords, d) in build_vars) <= 1,
                    name=f"OneDistPerTile_{coords[0]}_{coords[1]}")
# --- End Modification ---

# 2. Unique District Limit
# --- MODIFIED: Use build_vars dictionary access ---
for d in DISTRICTS_UNIQUE:
    model.addConstr(quicksum(build_vars[coords, d] for coords in tiles_workable if (coords, d) in build_vars) <= 1,
                    name=f"UniqueDistrict_{d}")
# --- End Modification ---

# 3. Mutual Exclusivity
print("DEBUG: Defining Mutual Exclusivity Constraints...")
for i, me_set in enumerate(DISTRICTS_MUTUALLY_EXCLUSIVE):
    print(f"  Processing ME set {i}: {me_set}")
    valid_me_districts = [d for d in me_set if d in ALL_DISTRICTS]
    print(f"  Valid districts in ME set: {valid_me_districts}")
    if valid_me_districts:
        # --- MODIFIED: Use build_vars dictionary access ---
        expr = quicksum(build_vars[coords, d]
                        for coords in tiles_workable
                        for d in valid_me_districts
                        if (coords, d) in build_vars) # Check key exists
        model.addConstr(expr <= 1, name=f"MutExcl_{i}")
        # --- End Modification ---
        print(f"  Added MutExcl_{i} constraint.")
    else:
            print(f"  Skipping ME set {i} as no valid districts found.")


# 4. Aqueduct Placement Requirements
AQUEDUCT = "Aqueduct"
if AQUEDUCT in ALL_DISTRICTS:
    # --- MODIFIED: Use build_vars dictionary access ---
    for coords in tiles_workable:
        key = (coords, AQUEDUCT)
        if key in build_vars: # Check if variable exists
            model.addConstr(build_vars[key] <= (1 if is_adjacent_to_cc.get(coords, False) else 0),
                            name=f"AqueductAdjCC_{coords[0]}_{coords[1]}")
            model.addConstr(build_vars[key] <= (1 if is_adjacent_to_freshwater.get(coords, False) else 0),
                            name=f"AqueductFreshwater_{coords[0]}_{coords[1]}")
    # --- End Modification ---

# 5. Harbor Placement Requirements
HARBOR = "Harbor"
if HARBOR in ALL_DISTRICTS:
        # --- MODIFIED: Use build_vars dictionary access ---
    for coords in tiles_workable:
            key = (coords, HARBOR)
            if key in build_vars: # Check if variable exists
                model.addConstr(build_vars[key] <= (1 if is_coast.get(coords, False) else 0),
                                name=f"HarborIsCoast_{coords[0]}_{coords[1]}")
                model.addConstr(build_vars[key] <= (1 if is_adjacent_to_land.get(coords, False) else 0),
                                name=f"HarborAdjLand_{coords[0]}_{coords[1]}")
        # --- End Modification ---

# 6. Dam Placement Requirements (Example - Needs refinement)
DAM = "Dam"
if DAM in ALL_DISTRICTS:
    # ... (Keep your existing Floodplains feature constraint) ...
    floodplains_feature_name = FEATURES.get("Floodplains", [])
    if floodplains_feature_name:
        for coords in tiles_workable:
                key = (coords, DAM)
                if key in build_vars:
                    tile_info = tile_data.get(coords)
                    is_floodplains = (tile_info is not None) and (tile_info.get('feature') in floodplains_feature_name)
                    model.addConstr(build_vars[key] <= (1 if is_floodplains else 0),
                                    name=f"DamPlacementFeature_{coords[0]}_{coords[1]}")
    else:
        print("Warning: Floodplains feature name not found in FEATURES constant. Skipping Dam placement constraint.")

    # --- NEW: Add constraint for river adjacency using the pre-calculated map ---
    print(f"  Adding Dam river requirement constraints...")
    dam_river_constraints_added = 0
    for coords in tiles_workable:
        key = (coords, DAM)
        if key in build_vars:
            # Dam variable can only be 1 if the tile it's placed on has a river edge
            is_river_tile = has_river_edge.get(coords, False)
            model.addConstr(build_vars[key] <= (1 if is_river_tile else 0),
                            name=f"DamPlacementRiver_{coords[0]}_{coords[1]}")
            dam_river_constraints_added += 1
    print(f"  Added {dam_river_constraints_added} Dam river constraints.")
    # --- End of new river constraint ---

print("Gurobi constraints defined.")

# --- Solve the LP Problem ---
print("\nSolving the optimization problem using Gurobi...")
model.optimize()

# --- Print the Results ---
print("\n--- Gurobi Optimization Results ---")
if model.Status == GRB.OPTIMAL:
    print(f"Status: Optimal")
    print(f"Optimal Total Adjacency Score = {model.ObjVal:.2f}")
    print("\nOptimal District Placements:")
    placed_districts = 0
    # Iterate through variables and check their solution values
    solution_placements = []
    # --- MODIFIED: Iterate through build_vars dictionary ---
    for key, var in build_vars.items():
        if var.X > 0.9: # Check if the variable is selected
                coords, district_name = key
                solution_placements.append((district_name, coords[0], coords[1]))
                placed_districts += 1
    # --- End Modification ---

    # Sort placements for readability (optional)
    solution_placements.sort()
    for placement in solution_placements:
        print(f"  Build {placement[0]} at ({placement[1]}, {placement[2]})")

    if placed_districts == 0:
        print("  No districts were placed in the optimal solution.")

elif model.Status == GRB.INFEASIBLE:
    print("Status: Infeasible - The problem has no solution.")
    print("Check constraints, data (especially workable tiles), and bonus values.")
    # Compute and print IIS (Irreducible Inconsistent Subsystem) to help debug
    print("\nComputing IIS to identify conflicting constraints...")
    try:
        model.computeIIS()
        model.write("civ6_infeasible.ilp")
        print("IIS written to civ6_infeasible.ilp")
    except GurobiError as iis_error: # Use the imported GurobiError
            print(f"Could not compute IIS: {iis_error}")

elif model.Status == GRB.UNBOUNDED:
    print("Status: Unbounded - The objective can be increased indefinitely.")
    print("This usually indicates missing constraints or an error in the objective function/coefficients.")
else:
    print(f"Status Code: {model.Status} (Not Optimal/Infeasible/Unbounded)")
    print("Solver did not find an optimal solution. Check Gurobi logs or status code meaning.")

Tiles considered workable before creating variables (29): [(53, 24), (53, 25), (54, 22), (54, 23), (54, 24), (54, 26), (55, 22), (55, 23), (55, 25), (55, 26), (55, 27), (56, 22), (56, 23), (56, 24), (56, 26), (56, 27), (56, 28), (57, 22), (57, 23), (57, 24), (57, 25), (57, 26), (57, 27), (57, 28), (58, 24), (58, 25), (58, 26), (58, 27), (59, 25)]
DEBUG: (57, 24) IS in tiles_workable before addVars.
Set parameter OutputFlag to value 1
Creating variables explicitly...
Gurobi variables defined (348 variables).
DEBUG: Key ((57, 24), 'Water Park') FOUND in build_vars after explicit creation.
******************************
DEBUG: Terrain bonus for (57, 24) and district Campus: 0
DEBUG: Terrain bonus for (57, 24) and district Holy Site: 0
DEBUG: Terrain bonus for (57, 24) and district Harbor: 0
DEBUG: Terrain bonus for (57, 24) and district Government Plaza: 0
DEBUG: Terrain bonus for (57, 24) and district Theater Square: 0
DEBUG: Terrain bonus for (57, 24) and district Entertainment Complex: